## LangChain-檢索增強生成(RAG)範例-OpenAI
2025/06/29 蘇彥庭

* 本篇範例主要建立中國四大名著的RAG
    * 四大名著文本[下載位置](https://github.com/tennessine/corpus)
    * 四大名著:《三國演義》、《西遊記》、《水滸傳》、《紅樓夢》
* Text-embeddings Model採用OpenAI API的`text-embedding-3-small`
* 聊天模型採用OpenAI API的`gpt-4o-mini`

In [1]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
import os

from dotenv import load_dotenv
load_dotenv()  # 讀取環境變數(OpenAI API Key)

True

In [2]:
# 設定文本切割器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

* 文本切割相關文章
    * [Langchain's Character Text Splitter - In-Depth Explanation](https://medium.com/@krishnahariharan/langchains-character-text-splitter-in-depth-explanation-5b0bf743121c)

In [3]:
# 讀取文本進行切分
# folder_path = "docs" 
# docs = []
# for f in os.listdir(folder_path):
#     file_path = os.path.join(folder_path, f)
#     if os.path.isfile(file_path) and f.endswith(".txt"):
#         loader = TextLoader(file_path, encoding="utf-8")
#         # 直接載入並切割後 append
#         docs.extend(text_splitter.split_documents(loader.load()))

filePath = "docs\三国演义.txt"
loader = TextLoader(filePath, encoding="utf-8")
docs = text_splitter.split_documents(loader.load())

print(f"已完成切割，共切出 {len(docs)} 個 chunks")

已完成切割，共切出 1749 個 chunks


In [4]:
# 顯示切割結果(取前5篇)
for i, doc in enumerate(docs[0:5]):
    print(f"【Chunk {i} - {doc.metadata.get('source')}】\n{doc.page_content}\n")

【Chunk 0 - docs\三国演义.txt】
《三国演义》罗贯中

第一回 宴桃园豪杰三结义 斩黄巾英雄首立功

    		

    滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。

    青山依旧在，几度夕阳红。白发渔樵江渚上，惯

    看秋月春风。一壶浊酒喜相逢。古今多少事，都付

    笑谈中。

    ——调寄《临江仙》

    话说天下大势，分久必合，合久必分。周末七国分争，并入于秦。及秦灭之后，楚、汉分争，又并入于汉。汉朝自高祖斩白蛇而起义，一统天下，后来光武中兴，传至献帝，遂分为三国。推其致乱之由，殆始于桓、灵二帝。桓帝禁锢善类，崇信宦官。及桓帝崩，灵帝即位，大将军窦武、太傅陈蕃共相辅佐。时有宦官曹节等弄权，窦武、陈蕃谋诛之，机事不密，反为所害，中涓自此愈横。

【Chunk 1 - docs\三国演义.txt】
建宁二年四月望日，帝御温德殿。方升座，殿角狂风骤起。只见一条大青蛇，从梁上飞将下来，蟠于椅上。帝惊倒，左右急救入宫，百官俱奔避。须臾，蛇不见了。忽然大雷大雨，加以冰雹，落到半夜方止，坏却房屋无数。建宁四年二月，洛阳地震；又海水泛溢，沿海居民，尽被大浪卷入海中。光和元年，雌鸡化雄。六月朔，黑气十余丈，飞入温德殿中。秋七月，有虹现于玉堂；五原山岸，尽皆崩裂。种种不祥，非止一端。帝下诏问群臣以灾异之由，议郎蔡邕上疏，以为蜺堕鸡化，乃妇寺干政之所致，言颇切直。帝览奏叹息，因起更衣。曹节在后窃视，悉宣告左右；遂以他事陷邕于罪，放归田里。后张让、赵忠、封谞、段珪、曹节、侯览、蹇硕、程旷、夏恽、郭胜十人朋比为奸，号为“十常侍”。帝尊信张让，呼为“阿父”。朝政日非，以致天下人心思乱，盗贼蜂起。

【Chunk 2 - docs\三国演义.txt】
时巨鹿郡有兄弟三人，一名张角，一名张宝，一名张梁。那张角本是个不第秀才，因入山采药，遇一老人，碧眼童颜，手执藜杖，唤角至一洞中，以天书三卷授之，曰：“此名《太平要术》，汝得之，当代天宣化，普救世人；若萌异心，必获恶报。”角拜问姓名。老人曰：“吾乃南华老仙也。”言讫，化阵清风而去。角得此书，晓夜攻习，能呼风唤雨，号为“太平道人”。中平元年正月内，疫气流行，张角散施符水，为人治病，自称“大贤良师”。角有徒弟五百余人，云游四方，皆能书符念咒。次后徒众日多，角乃立三十六方，大方万余人，小方六七千，各立

In [13]:
# 建立 Embeddings
embedding_model_name = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(model=embedding_model_name)

In [9]:
# Chunk做Embeddings的範例(取前5篇)
texts = [doc.page_content for doc in docs[0:5]]
vectors = embeddings.embed_documents(texts)
for i in range(len(texts)):
    print(f"\n=== Chunk {i+1} ===")
    print(texts[i][:200])   # 顯示 chunk 前 200 字
    print(f"向量長度: {len(vectors[i])}")
    print(f"向量前 10 維: {vectors[i][:10]}")


=== Chunk 1 ===
《三国演义》罗贯中

第一回 宴桃园豪杰三结义 斩黄巾英雄首立功

    		

    滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。

    青山依旧在，几度夕阳红。白发渔樵江渚上，惯

    看秋月春风。一壶浊酒喜相逢。古今多少事，都付

    笑谈中。

    ——调寄《临江仙》

    话说天下大势，分久必合，合久必分。周末七国分争，并入于秦。及秦灭之后，楚、汉分争，又并入于
向量長度: 1536
向量前 10 維: [-0.021732984110713005, 0.0007308850181289017, 0.030244624242186546, -0.010845132172107697, 0.01732230931520462, -0.033576659858226776, -0.022598031908273697, 0.016190271824598312, -0.016521338373422623, 0.03060773015022278]

=== Chunk 2 ===
建宁二年四月望日，帝御温德殿。方升座，殿角狂风骤起。只见一条大青蛇，从梁上飞将下来，蟠于椅上。帝惊倒，左右急救入宫，百官俱奔避。须臾，蛇不见了。忽然大雷大雨，加以冰雹，落到半夜方止，坏却房屋无数。建宁四年二月，洛阳地震；又海水泛溢，沿海居民，尽被大浪卷入海中。光和元年，雌鸡化雄。六月朔，黑气十余丈，飞入温德殿中。秋七月，有虹现于玉堂；五原山岸，尽皆崩裂。种种不祥，非止一端。帝下诏问群臣以灾异之由，
向量長度: 1536
向量前 10 維: [0.015814125537872314, 0.03363639488816261, 0.02580920048058033, 0.015323500148952007, -0.021074090152978897, 0.0034600531216710806, -0.04452143982052803, 0.016989344730973244, -0.07133471220731735, 0.0001968208234757185]

=== Chunk 3 ===
时巨鹿郡有兄弟三人，一名张角，一名张宝，一名张梁。那张角本是个不第秀才，因入山采药，遇一老人，碧眼童颜，手执藜杖，唤

In [14]:
# 建立向量資料庫
persist_directory = "./chroma_db_openai"
if os.path.exists(os.path.join(persist_directory, "index")):
    print("偵測到已有 Chroma 向量資料庫，直接載入...")
    db = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )
else:
    print("未偵測到 Chroma 向量資料庫，建立新向量資料庫...")
    # db = Chroma.from_documents(
    #     docs,
    #     embeddings,
    #     persist_directory=persist_directory
    # )
    
    db = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )

    # 受限於request token限制 採分批處理
    batch_size = 50
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i+batch_size]
        try:
            db.add_documents(batch)
        except Exception as e:
            print(f"處理批次 {i} 至 {i+batch_size} 時發生錯誤: {e}")
            continue

C:\Users\User\AppData\Local\Temp\ipykernel_27392\679699976.py:17: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


未偵測到 Chroma 向量資料庫，建立新向量資料庫...


In [15]:
print(f"向量資料庫目前筆數: {db._collection.count()}")

向量資料庫目前筆數: 1749


In [16]:
# 設定檢索器
# k: 每次檢索時 從向量資料庫中取回最相關的 Top-k 筆文件段落(chunks)
retriever = db.as_retriever(search_kwargs={"k": 5})

In [17]:
# 建立 RAG QA Chain
llm_model_name = "gpt-4o-mini"
llm = ChatOpenAI(
    model=llm_model_name,
    temperature=0.2
)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 將所有檢索到的段落「直接拼接」在一起，與問題一起輸入 LLM（最簡單、適合段落少時使用）
    retriever=retriever,
    return_source_documents=True
)

* RetrievalQA的chain_type參考文章
    * [Using langchain for Question Answering on Own Data](https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed)  

In [20]:
# 提問 & 回覆
# query = "請問劉備底下有哪些武將?請用繁體中文回答。"
# query = "張飛的武器是什麼?請用繁體中文回答。"
# query = "誰對曹操的評價是'治世之能臣，亂世之奸雄'?請用繁體中文回答。"
query = "誰執行草船借箭的策略?請用繁體中文回答。"
result = qa(query)

print("\n=== 回覆 ===")
print(result["result"])

print("\n=== 檢索來源 (僅呈現前300字) ===")
for idx, doc in enumerate(result["source_documents"]):
    print(f"\n--- Source {idx+1} ({doc.metadata.get('source')}) ---")
    print(doc.page_content[:300])


=== 回覆 ===
執行草船借箭的策略的是諸葛亮。

=== 檢索來源 (僅呈現前300字) ===

--- Source 1 (docs\三国演义.txt) ---
“大江之中，潮生潮落，风浪不息；北兵不惯乘舟，受此颠播，便生疾病。若以大船小船各皆配搭，或三十为一排，或五十为一排，首尾用铁环连锁，上铺阔板，休言人可渡，马亦可走矣，乘此而行，任他风浪潮水上下，复何惧哉？”曹操下席而谢曰：“非先生良谋，安能破东吴耶！”统曰：“愚浅之见，丞相自裁之。”操即时传令，唤军中铁匠，连夜打造连环大钉，锁住船只。诸军闻之，俱各喜悦。后人有诗曰：“赤壁鏖兵用火攻，运筹决策尽皆同。若非庞统连环计，公瑾安能立大功？”

--- Source 2 (docs\三国演义.txt) ---
船出。聘立于船头，大叫：“丞相钧旨：南船且休近寨，就江心抛住。”众军齐喝：“快下了篷！”言未绝，弓弦响处，文聘被箭射中左臂，倒在船中。船上大乱，各自奔回。南船距操寨止隔二里水面。黄盖用刀一招，前船一齐发火。火趁风威，风助火势，船如箭发，烟焰涨天。二十只火船，撞入水寨，曹寨中船只一时尽着；又被铁环锁住，无处逃避。隔江炮响，四下火船齐到，但见三江面上，火逐风飞，一派通红，漫天彻地。

--- Source 3 (docs\三国演义.txt) ---
操升帐谓众谋士曰：“若非天命助吾，安得凤雏妙计？铁索连舟，果然渡江如履平地。”程昱曰：“船皆连锁，固是平稳；但彼若用火攻，难以回避。不可不防。”操大笑曰：“程仲德虽有远虑，却还有见不到处。”荀攸曰：“仲德之言甚是。丞相何故笑之？”操曰：“凡用火攻，必藉风力。方今隆冬之际，但有西风北风，安有东风南风耶？吾居于西北之上，彼兵皆在南岸，彼若用火，是烧自己之兵也，吾何惧哉？若是十月小春之时，吾早已提备矣。”诸将皆拜伏曰：“丞相高见，众人不及。”操顾诸将曰：“青、徐、燕、代之众，不惯乘舟。今非此计，安能涉大江之险！”只见班部中二将挺身出曰：“小将虽幽、燕之人，也能乘舟。今愿借巡船二十只，直至江口，夺旗鼓

--- Source 4 (docs\三国演义.txt) ---
且说江东，天色向晚，周瑜唤出蔡和，令军士缚倒。和叫：“无罪！”瑜曰：“汝是何等人，敢来诈降！吾今缺少福物祭旗，愿借你首级。”和抵赖不过，大叫曰：“汝家阚泽、甘宁亦曾与谋！”瑜曰：“此乃吾之所使也。”蔡